# Erosion Detection Training

This notebook downloads a sample dataset, adds synthetic sensor data, computes correlations between image features and sensors, and trains a simple model.


In [ ]:
# Install dependencies if running on a fresh environment
!pip install -q -r ../requirements.txt


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Download CIFAR10 as a placeholder dataset
transform = transforms.Compose([transforms.ToTensor()])
train_ds = datasets.CIFAR10(root='data', train=True, download=True, transform=transform)


In [ ]:
# Create synthetic sensor data correlated with class label
np.random.seed(0)
labels = np.array(train_ds.targets)
sensors = pd.DataFrame({
    'rainfall': labels * 10 + np.random.randn(len(labels)) * 2,
    'slope': labels * 5 + np.random.randn(len(labels)) * 1.5,
    'humidity': labels * 8 + np.random.randn(len(labels)) * 1.0,
})


In [ ]:
# Correlate sensors with labels
correlation = sensors.corrwith(pd.Series(labels, name='label'))
print(correlation)


In [ ]:
# Create a dataset that includes images and sensors
class SensorImageDataset(Dataset):
    def __init__(self, image_ds, sensor_df):
        self.image_ds = image_ds
        self.sensor_df = sensor_df.reset_index(drop=True)
    def __len__(self):
        return len(self.image_ds)
    def __getitem__(self, idx):
        image, label = self.image_ds[idx]
        sensors = torch.tensor(self.sensor_df.iloc[idx].values, dtype=torch.float32)
        return image, sensors, label

full_ds = SensorImageDataset(train_ds, sensors)
loader = DataLoader(full_ds, batch_size=64, shuffle=True)


In [ ]:
# Simple model combining CNN features with sensor data
import torch.nn as nn
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc_image = nn.Linear(32, 16)
        self.fc_sensor = nn.Linear(3, 16)
        self.classifier = nn.Linear(32, 10)
    def forward(self, x_img, x_sensor):
        x = self.cnn(x_img)
        x = x.view(x.size(0), -1)
        x_img_feat = self.fc_image(x)
        x_sensor_feat = self.fc_sensor(x_sensor)
        x = torch.cat([x_img_feat, x_sensor_feat], dim=1)
        return self.classifier(x)
model = SimpleModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
# Train for a few epochs
for epoch in range(2):
    for images, sensor_values, labels in loader:
        optimizer.zero_grad()
        outputs = model(images, sensor_values)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1} complete, loss: {loss.item():.4f}')


Training complete.